In [1]:
pip install tensorflow mtcnn opencv-python-headless matplotlib


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import time
import numpy as np
from mtcnn import MTCNN
import matplotlib.pyplot as plt
detector = MTCNN()
video_capture = cv2.VideoCapture(0)
desired_fps = 10  
frame_delay = 1 / desired_fps  
padding_factor = 0.2  
while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb_frame)
    for face in faces:
        x, y, width, height = face['box']
        x, y = abs(x), abs(y)  
        x_pad = int(width * padding_factor)  
        y_pad = int(height * padding_factor)  
        x_new = max(0, x - x_pad)
        y_new = max(0, y - y_pad)
        width_new = width + 3 * x_pad
        height_new = height + 3 * y_pad
        face_region = rgb_frame[y_new:y_new + height_new, x_new:x_new + width_new]
        gray_face = cv2.cvtColor(face_region, cv2.COLOR_RGB2GRAY)
        blurred_face = cv2.GaussianBlur(gray_face, (5, 5), 1.4)
        edges = cv2.Canny(blurred_face, threshold1=100, threshold2=200)
        colored_edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        frame[y_new:y_new + height_new, x_new:x_new + width_new] = cv2.cvtColor(colored_edges, cv2.COLOR_RGB2BGR)
        cv2.rectangle(frame, (x_new, y_new), (x_new + width_new, y_new + height_new), (0, 255, 0), 2)
    cv2.imshow('Face Detection with Canny Edges (TensorFlow + MTCNN)', frame)
    time.sleep(frame_delay)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()
